In [1]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DOWNLOAD_DIR) / f'df_OHLCV_{date_str}.parquet'
dest_path = Path(DEST_DIR) / f'df_OHLCV_{date_str}_clean.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-02
DOWNLOAD_DIR: C:\Users\ping\Downloads
DEST_DIR: ..\data

source_path: C:\Users\ping\Downloads\df_OHLCV_2025-04-02.parquet
dest_path: ..\data\df_OHLCV_2025-04-02_clean.parquet


In [2]:
# --- Data Loading & Initial Inspection ---
import pandas as pd

# # Load raw data from pickle file
# df = pd.read_pickle(source_path)

df = pd.read_parquet(source_path, engine='pyarrow')

# Display initial data structure
print("[Raw Data Overview]")
display(df.head())
df.info()

[Raw Data Overview]


Open   High    Low  Close  Adj Close   Volume  Adj Open  \
Symbol Date                                                                   
WMB    2025-04-02  59.91  61.65  59.91  61.60      61.60  5142727     59.91   
       2025-04-01  59.53  60.61  59.16  60.57      60.57  5774800     59.53   
       2025-03-31  58.70  59.99  58.60  59.76      59.76  8266900     58.70   
       2025-03-28  59.36  59.48  58.67  59.19      59.19  5451600     59.36   
       2025-03-27  59.75  59.96  59.00  59.43      59.43  5421300     59.75   

                   Adj High  Adj Low  
Symbol Date                           
WMB    2025-04-02     61.65    59.91  
       2025-04-01     60.61    59.16  
       2025-03-31     59.99    58.60  
       2025-03-28     59.48    58.67  
       2025-03-27     59.96    59.00

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 389295 entries, ('WMB', Timestamp('2025-04-02 00:00:00')) to ('PSO', Timestamp('2024-04-03 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Open       389295 non-null  float64
 1   High       389295 non-null  float64
 2   Low        389295 non-null  float64
 3   Close      389295 non-null  float64
 4   Adj Close  389295 non-null  float64
 5   Volume     389275 non-null  Int64  
 6   Adj Open   389295 non-null  float64
 7   Adj High   389295 non-null  float64
 8   Adj Low    389295 non-null  float64
dtypes: Int64(1), float64(8)
memory usage: 28.6+ MB


In [3]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))


# --- Data Filtering & Cleaning ---
import utils  # Custom utility functions

# 1. Align dates across all symbols using SPY as reference
df = utils.filter_df_dates_to_reference_symbol(df=df, reference_symbol='SPY')

# 2. Remove symbols with missing data points
df_clean, missing_symbols = utils.filter_symbols_with_missing_values(df)

# Display cleaning results
print("\n[Cleaning Report]")
print(f"Removed {len(missing_symbols)} symbols with missing data: {missing_symbols}")
print("\n[Cleaned Data Structure]")
df_clean.info()

Original number of symbols: 1559
Number of symbols after filtering: 1538
Number of symbols filtered out: 21

First 10 symbols that were filtered out:
['WAY', 'RBRK', 'LINE', 'SFD', 'ULS', 'STRK', 'SARO', 'ZK', 'LTM', 'SW']

Example of dates for first filtered out symbol:

Dates for WAY:
DatetimeIndex(['2025-04-02', '2025-04-01', '2025-03-31', '2025-03-28',
               '2025-03-27', '2025-03-26', '2025-03-25', '2025-03-24',
               '2025-03-21', '2025-03-20',
               ...
               '2024-06-21', '2024-06-20', '2024-06-18', '2024-06-17',
               '2024-06-14', '2024-06-13', '2024-06-12', '2024-06-11',
               '2024-06-10', '2024-06-07'],
              dtype='datetime64[ns]', name='Date', length=205, freq=None)

Filtered DataFrame info:
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 386038 entries, ('WMB', Timestamp('2025-04-02 00:00:00')) to ('PSO', Timestamp('2024-04-03 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtyp

In [4]:
# --- Save Cleaned Data ---
# # Save processed data to pickle file
# df_clean.to_pickle(dest_path)

# Using PyArrow (default, recommended for most cases)
df_clean.to_parquet(dest_path, engine='pyarrow')
print(f"\n[Save Successful] Cleaned data saved to:\n{dest_path}")




[Save Successful] Cleaned data saved to:
..\data\df_OHLCV_2025-04-02_clean.parquet
